In [1]:
import json
import requests
from PIL import Image
from tensorflow.keras.utils import img_to_array
import numpy as np
import tensorflow as tf

def prepare_image(src_img, target):
    if src_img.mode != "RGB":
        src_img = src_img.convert("RGB")
    src_img = src_img.resize(target)

    src_img = img_to_array(src_img)
    src_img = np.expand_dims(src_img, axis=0)

    return src_img

pb_path = "./model_saved/tf/eyes/efficientnet_5/efficientnet.h5"
model = tf.keras.models.load_model(pb_path)

2023-05-02 16:35:01.833716: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


2023-05-02 16:35:01.846619: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-02 16:35:01.846971: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-02 16:35:01.848011: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 16:35:01.848985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

In [ ]:
!git add /home/jupyter/pet_disease/

In [ ]:
!git commit -m "날짜 + acc + val_acc 등의 정보를 커밋 할 것임"

In [ ]:
!git push